# 人名列挙の探索

Date: 2023/12/23-2023/12/25

In [1]:
import spacy

In [2]:
nlp = spacy.load('en_core_web_trf')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
text1 = "The most renowned Clavier composers of that day were Froberger,50 Fischer,51 Johann Caspar Kerl,52 Pachelbel,53 Buxtehude,54 Bruhns,55 [pg 11]and Böhm.56 Johann Christoph possessed a book containing several pieces by these masters, and Bach begged earnestly for it, but without effect. Refusal increasing his determination, he laid his plans to get the book without his brother's knowledge. It was kept on a book-shelf which had a latticed front. Bach's hands were small. Inserting them, he got hold of the book, rolled it up, and drew it out. As he was not allowed a candle, he could only copy it on moonlight nights, and it was six months before he finished his heavy task. As soon as it was completed he looked forward to using in secret a treasure won by so much labour. But his brother found the copy and took it from him without pity, nor did Bach recover it until his brother's death soon after."
text2 = "In 1802, for reasons which he explains in his Preface, Forkel published from Hoffmeister and Kühnel's “Bureau de Musique” his Ueber Johann Sebastian Bachs Leben, Kunst und Kunstwerke. Für patriotische Verehrer echter musikalischer Kunst, of which a new edition was issued by Peters in 1856. The original edition bears a dedication to Gottfried Baron van Swieten1 (1734-1803), Prefect of the Royal Library, Vienna, and sometime Austrian Ambassador in Berlin, a friend of Haydn and Mozart, patron of Beethoven, a man whose age allowed him to have seen Bach, and whose career makes the association with Bach that Forkel's dedication gives him not undeserved. It was he, an ardent Bach enthusiast, who introduced the youthful Mozart to the music of the Leipzig Cantor. “I go every Sunday at twelve o'clock to the Baron van Swieten,” Mozart writes in 1782, “where nothing is played but Handel and [pg xiii]Bach, and I am now making a collection of the Fugues of Bach.” The merit and limitations of Forkel's book will be considered later. For the moment the fact deserves emphasis that, inadequate as it is, it presented a fuller picture of Bach than so far had been drawn, and was the first to render the homage due to his genius."
text = text1 + '\n\n' + text2
print(text)

The most renowned Clavier composers of that day were Froberger,50 Fischer,51 Johann Caspar Kerl,52 Pachelbel,53 Buxtehude,54 Bruhns,55 [pg 11]and Böhm.56 Johann Christoph possessed a book containing several pieces by these masters, and Bach begged earnestly for it, but without effect. Refusal increasing his determination, he laid his plans to get the book without his brother's knowledge. It was kept on a book-shelf which had a latticed front. Bach's hands were small. Inserting them, he got hold of the book, rolled it up, and drew it out. As he was not allowed a candle, he could only copy it on moonlight nights, and it was six months before he finished his heavy task. As soon as it was completed he looked forward to using in secret a treasure won by so much labour. But his brother found the copy and took it from him without pity, nor did Bach recover it until his brother's death soon after.

In 1802, for reasons which he explains in his Preface, Forkel published from Hoffmeister and Küh

In [4]:
import re
text = re.sub(r'\[.*\]|\d?', '', text)
print(text)

The most renowned Clavier composers of that day were Froberger, Fischer, Johann Caspar Kerl, Pachelbel, Buxtehude, Bruhns, and Böhm. Johann Christoph possessed a book containing several pieces by these masters, and Bach begged earnestly for it, but without effect. Refusal increasing his determination, he laid his plans to get the book without his brother's knowledge. It was kept on a book-shelf which had a latticed front. Bach's hands were small. Inserting them, he got hold of the book, rolled it up, and drew it out. As he was not allowed a candle, he could only copy it on moonlight nights, and it was six months before he finished his heavy task. As soon as it was completed he looked forward to using in secret a treasure won by so much labour. But his brother found the copy and took it from him without pity, nor did Bach recover it until his brother's death soon after.

In , for reasons which he explains in his Preface, Forkel published from Hoffmeister and Kühnel's “Bureau de Musique”

In [5]:
paragraphes = text.split('\n\n')
paragraphes

["The most renowned Clavier composers of that day were Froberger, Fischer, Johann Caspar Kerl, Pachelbel, Buxtehude, Bruhns, and Böhm. Johann Christoph possessed a book containing several pieces by these masters, and Bach begged earnestly for it, but without effect. Refusal increasing his determination, he laid his plans to get the book without his brother's knowledge. It was kept on a book-shelf which had a latticed front. Bach's hands were small. Inserting them, he got hold of the book, rolled it up, and drew it out. As he was not allowed a candle, he could only copy it on moonlight nights, and it was six months before he finished his heavy task. As soon as it was completed he looked forward to using in secret a treasure won by so much labour. But his brother found the copy and took it from him without pity, nor did Bach recover it until his brother's death soon after.",
 "In , for reasons which he explains in his Preface, Forkel published from Hoffmeister and Kühnel's “Bureau de Mus

In [6]:
from spacy import displacy

text = "The most renowned Clavier composers of that day were Froberger, Fischer, Johann Caspar Kerl, Pachelbel, Buxtehude, Bruhns, and Böhm."
doc = nlp(text)
displacy.render(doc, style='dep')

In [7]:
# グループの先頭のTokenを探索
def findHead(token, head):
    if token.dep_ == 'conj':
        findHead(token.head, head)
    else:
        head.append(token)

# PROPNで列挙されるTokenを探索
def findGroup(token, groups):
    for t in token.children:
        if (t.pos_ == 'PROPN' and t.dep_ == 'conj'):
            #print(t.head.text, t.text)
            head = []
            findHead(t, head)
            #print(head)
            head_ = head[0]
            if not head_ in groups:
                groups[head_] = [head_]
            groups[head_].append(t)
        try:
            first = next(t.children)
            findGroup(t, groups)
        except StopIteration:
            pass

# フルネームを得るため、compoundの関係で結ばれるPROPNを全て接続
def fullname(token, text=False):
    name = []
    for t in token.children:
        if t.dep_ == 'compound':
            name.append(t)
    name.append(token)
    if text:
        return ' '.join([token.text for token in name])
    else:
        return name

# 文の中で名前の列挙を探索し列挙グループの配列と列挙グループに含まれない名前の配列を返す
def findPersonsInGroup(doc):
    result = []
    persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            groups = {}
            findGroup(root, groups)
            for group in groups.values():
                fullname_list = [fullname(token, text=True) for token in group]
                persons_fullname_list = [name for name in fullname_list if name in persons]
                #print(persons_fullname_list)
                if len(persons_fullname_list) > 0:
                    result.append(persons_fullname_list)
    for group in result:
        for name in group:
            persons.remove(name)
    return (result, persons)

In [8]:
for p in paragraphes:
    sents = nlp(p).sents
    for doc in nlp.pipe([sent.text for sent in sents]):
        print(f'Doc: {doc.text}')
        result, persons = findPersonsInGroup(doc)
        print(f'Groups: {result}, \nPersons: {persons}', end='\n\n')

Doc: The most renowned Clavier composers of that day were Froberger, Fischer, Johann Caspar Kerl, Pachelbel, Buxtehude, Bruhns, and Böhm.
Groups: [['Froberger', 'Fischer', 'Johann Caspar Kerl', 'Pachelbel', 'Buxtehude', 'Bruhns', 'Böhm']], 
Persons: []

Doc: Johann Christoph possessed a book containing several pieces by these masters, and Bach begged earnestly for it, but without effect.
Groups: [], 
Persons: ['Johann Christoph', 'Bach']

Doc: Refusal increasing his determination, he laid his plans to get the book without his brother's knowledge.
Groups: [], 
Persons: []

Doc: It was kept on a book-shelf which had a latticed front.
Groups: [], 
Persons: []

Doc: Bach's hands were small.
Groups: [], 
Persons: ['Bach']

Doc: Inserting them, he got hold of the book, rolled it up, and drew it out.
Groups: [], 
Persons: []

Doc: As he was not allowed a candle, he could only copy it on moonlight nights, and it was six months before he finished his heavy task.
Groups: [], 
Persons: []

Doc: A